In [1]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.request import urlopen
import time
import re
import pandas as pd

In [20]:
#scrapping hackernoon sections:

url = 'https://hackernoon.com/tagged/'
feat_list = ['cryptocurrency','coding','artificial-intelligence',
             'futurist','startups']

lista_articulos = []
for features in feat_list:
    res = requests.get(url+features)
    page = res.content
    soup = bs(page, 'html.parser')
    for article in soup.find_all('div',{'class':'title'}):
        news = article.find('a', href=True)
        lista_articulos.append('https://hackernoon.com' + news.get('href'))


In [3]:
#get title, body and tags of each article


tag_dict = dict()
title_list = []
text_list=[]
counter=0
counter2=1
counter3=0
counter4=1
start=time.time()
for url in lista_articulos:
    
    paragraph = []
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    
    try:
        for t in soup.find_all('div',{'class':'paragraph'}):
            paragraph.append(t.text)
        text_list.append(' '.join(paragraph))
        
    except:
        raise ValueError
  
    try:   
        
        for title in soup.find('h1', {'class':'title'}):
            if title in title_list:
                title_list.append(title+str(counter2))
                counter2+=1
            else:
                title_list.append(title)
    except:
        
        title = "No title" + str(counter)
        counter+=1
        title_list.append(title)

    tags = []
    
    for i in soup.find_all('div', {'class':'archive-tags'}):
        tags_str=i.text.split('\n')[1:-1]
    tags.append(tags_str)   
    counter4+=1
    tag_dict[str(counter4)]=tags

end = time.time()
print(end-start)
    

282.92196130752563


In [4]:
#regex to clean text format
def regexTitle(s):
    lst=[]
    for text in s:
        lst.append(re.sub('[âÂ]\S*','', text))
    return lst

title_list=regexTitle(title_list)
text_list=regexTitle(text_list)
    

In [5]:
articles_dict = {"id": lista_articulos, "title": title_list, "text": text_list}


In [6]:
#create dataframe from gathered data

articles = pd.DataFrame.from_dict(articles_dict)
articles['tags'] = tag_dict.values()
articles

,id,title,text,tags
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...","([Blockchain, Bitcoin, Cryptocurrency, Educati..."
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,"([Blockchain, Merkle Tree, Data Structures, Cr..."
2,https://hackernoon.com/data-signals-vs-noise-m...,Data Signals vs. Noise: Misleading Metrics and...,The steady growth in the crypto-asset space ha...,"([Cryptocurrency, Bitcoin, Ethereum, Data Scie..."
3,https://hackernoon.com/a-crypto-traders-diary-...,A crypto-trader diary week 1,I have decided that I want to become a person ...,"([Trading, Crypto, Cryptocurrency, Bitcoin, Cr..."
4,https://hackernoon.com/understanding-cryptocur...,Understanding Cryptocurrency Development,"more that you read, the more things you will...","([Blockchain, Cryptocurrency, Software Develop..."
5,https://hackernoon.com/how-irrational-exuberan...,How Irrational Exuberance aligned Dot-Com and ...,Nearly all markets at some point go through a ...,"([Venture Capital, Cryptocurrency, Cryptocurre..."
6,https://hackernoon.com/security-tokens-the-new...,Security tokens: The New Blockchain Era,Blockchain is evolving into a new phase of our...,"([Blockchain, Security Tokens, Security Tokeno..."
7,https://hackernoon.com/rethinking-the-identity...,Ethereum.org's New Art: \nRethinking Identity ...,"Today, I am excited to launch the first new ar...","([Ethereum, Blockchain, Cryptocurrency, Eth, D..."
8,https://hackernoon.com/what-indias-potential-c...,What Does India's Potential Crypto Ban Mean fo...,India long-standing aversion to cryptocurrenci...,"([Cryptocurrency, Startup, India, Reserve Bank..."
9,https://hackernoon.com/the-ultimate-guide-to-u...,The Ultimate Guide To Understanding EOS Accounts,EOS uses a different account structure than mo...,"([Blockchain, Eos, Eosio, Cryptocurrency, Eos ..."


In [7]:
#clean tags columns: get a list, not a tuple.
articles.tags = [e[0] for e in articles.tags]

In [8]:
#fix problem with texts. The scraping has only been effective for one tag('paragraph'), but some articles do not hare that label. Instead, they use the 'p' classifier.
#we opt for dropping those rows which do not contain text

articles = articles[articles.text != '']
articles

,id,title,text,tags
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...","[Blockchain, Bitcoin, Cryptocurrency, Education]"
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,"[Blockchain, Merkle Tree, Data Structures, Cry..."
2,https://hackernoon.com/data-signals-vs-noise-m...,Data Signals vs. Noise: Misleading Metrics and...,The steady growth in the crypto-asset space ha...,"[Cryptocurrency, Bitcoin, Ethereum, Data Scien..."
3,https://hackernoon.com/a-crypto-traders-diary-...,A crypto-trader diary week 1,I have decided that I want to become a person ...,"[Trading, Crypto, Cryptocurrency, Bitcoin, Cry..."
4,https://hackernoon.com/understanding-cryptocur...,Understanding Cryptocurrency Development,"more that you read, the more things you will...","[Blockchain, Cryptocurrency, Software Developm..."
5,https://hackernoon.com/how-irrational-exuberan...,How Irrational Exuberance aligned Dot-Com and ...,Nearly all markets at some point go through a ...,"[Venture Capital, Cryptocurrency, Cryptocurren..."
6,https://hackernoon.com/security-tokens-the-new...,Security tokens: The New Blockchain Era,Blockchain is evolving into a new phase of our...,"[Blockchain, Security Tokens, Security Tokenom..."
7,https://hackernoon.com/rethinking-the-identity...,Ethereum.org's New Art: \nRethinking Identity ...,"Today, I am excited to launch the first new ar...","[Ethereum, Blockchain, Cryptocurrency, Eth, De..."
8,https://hackernoon.com/what-indias-potential-c...,What Does India's Potential Crypto Ban Mean fo...,India long-standing aversion to cryptocurrenci...,"[Cryptocurrency, Startup, India, Reserve Bank ..."
9,https://hackernoon.com/the-ultimate-guide-to-u...,The Ultimate Guide To Understanding EOS Accounts,EOS uses a different account structure than mo...,"[Blockchain, Eos, Eosio, Cryptocurrency, Eos A..."


In [9]:
def tagSeparator(df):
    s = df.tags.apply(pd.Series,1).stack()
    s.index=s.index.droplevel(-1)
    s.name = 'Tags'
    return df.join(s)

new_articles= tagSeparator(articles)

new_articles.drop('tags', axis=1, inplace=True)
type(new_articles)


pandas.core.frame.DataFrame

In [15]:
new_articles.Tags.unique()

array(['Blockchain', 'Bitcoin', 'Cryptocurrency', 'Education',
       'Merkle Tree', 'Data Structures', 'Latest Tech Stories',
       'Hackernoon Top Story', 'Wtf Is A Merkle Tree',
       'What Is A Merkle Tree', 'Ethereum', 'Data Science',
       'Machine Learning', 'Deep Learning', 'Intotheblock', 'Trading',
       'Crypto', 'Cryptocurrency Investment', 'Software Development',
       'Cryptodevelopment', 'Blockchain Development',
       'Cryptocurrency Developmment', 'Understanding Cryptocurrency',
       'Understanding Crypto Dev', 'Venture Capital', 'Finance', 'Vc',
       'Crypto Bubbles', 'Irrational Exuberance', 'Security Tokens',
       'Security Tokenomics', 'Latest Tech Story', 'Blockchain Era',
       'Blockchain Evolution', 'Utility Tokens', 'Eth', 'Design',
       'Dialect', 'The New Octahedron', 'Startup', 'India',
       'Reserve Bank Of India', 'Cryptocurrency India',
       'Cryptocurrency Ban', 'Cryptocurrency Ban India', 'Eos', 'Eosio',
       'Eos Account', 'Eos Wa

In [16]:
drop_tags = ['Top tech stories', 'Latest','Hacker noon awards', 'Tech awards','Noonies 2019',
                  'Hackernoon medium', 'Medium hackernoon', 'David smooke',
       'Ev williams', 'Hackernoon', 'Medium','Killing globalization',
       'Killing globalisation', 'I see the world','Wtf Is A Merkle Tree','What Is A Merkle Tree',
       'Another tech thought', 'Founders', 'Founder stories',
      'Problemeter','Latet tech stories', 'Promiseallsettled', 'Promiseany',' all encompassing enemy',
                 'Be for something', 'Erik Brynjolfsson']

new_articles[~new_articles.Tags.isin(drop_tags)]


,id,title,text,Tags
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...",Blockchain
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...",Bitcoin
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...",Cryptocurrency
0,https://hackernoon.com/whats-the-best-smartpho...,What the Best Smartphone for keeping your Conv...,"In the world of today, personal data like our ...",Education
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Blockchain
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Merkle Tree
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Data Structures
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Cryptocurrency
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Latest Tech Stories
1,https://hackernoon.com/blockchain-what-the-hel...,Blockchain: What the Hell is a Merkle Tree?,No you can find this kind of a tree in the for...,Hackernoon Top Story


In [18]:
#load dataset

new_articles.to_csv('text_classifier.csv')